In [1]:
import pandas as pd
import numpy as np



### Get all the pillar names from the excel

In [2]:
ls ../..

Invalid switch - "..".


In [3]:
names = pd.read_excel('../../UNDP Digital Assessment Data Framework Filename Matching V7.xlsx')

In [4]:
col_names = ['Indicator','check', 'Data Source','Index','Filename']

In [5]:
names = names[col_names]

In [6]:
names.head()

,Indicator,check,Data Source,Index,Filename
0,Countries,NaN,United Nations,False,Countries
1,"Database of Global Administrative Areas (GADM,...",NaN,GADM maps and data,False,NaN
2,High Resolution Population Density Maps + Demo...,NaN,Facebook,False,NaN
3,population density vs openstreetmap object den...,NaN,Kontur,False,NaN
4,Population Density,Infrastructure,World Bank,False,population_density


In [7]:
# get all the files per pillar
data_stats = names.groupby('check').agg({'Filename':'count','Indicator':'count'})

In [8]:
data_stats

,Filename,Indicator
check,,
Business,20,25
Foundations,9,12
Government,10,15
Infrastructure,41,48
People,39,47
Regulation,6,7
Strategy,1,1


### Foundations

In [9]:
bnames = names[(names.check=='Government')&(~names.Filename.isna())]#&(names.Index==False)]

In [10]:
bnames.head(25)

,Indicator,check,Data Source,Index,Filename
53,Online-Service-Index (OSI),Government,DESA,True,e_government_index
54,E-Participation index,Government,DESA,True,e_government_index
55,Use of public services online (% of services o...,Government,Boston Consulting Group/SalesForce,False,digital_public_service_use
56,What is the % change of government digitizing ...,Government,World Bank,TRUE,Egov_strategy
57,Security incidents (# of relevant issues),Government,SPECOPS/CSIS,False,cyber_attacks
58,ICT investment as a percentage of GDP,Government,OECD,False,ICT_Investment
59,R&D spending (% of GDP),Government,World Bank,False,RD_Percentage_GDP
60,Evidence of digital strategies in/across Minst...,Government,World Bank,False,Egov_strategy
62,Evidence of focus on vulnerable groups,Government,World Bank,False,Egov_strategy
65,% of digital skills certifications / training ...,Government,Coursera,False,digital_skill_level


In [11]:
# get list of names for all indicators
indicators = bnames.Indicator.unique()

In [12]:
# get all file names
bfiles = bnames.Filename.unique()

In [13]:
bfiles

array(['e_government_index', 'digital_public_service_use',
       'Egov_strategy', 'cyber_attacks', 'ICT_Investment',
       'RD_Percentage_GDP', 'digital_skill_level'], dtype=object)

In [14]:
# formula for converting scale
def convert_rank(old_value, old_min=1, old_max=7, new_min=1, new_max=6 ):
    """ Convert old scale values scale into new scale values"""
    old_range = old_max - old_min
    new_range = new_max - new_min
    new_value = (((old_value-old_min)*new_range)/old_range)+new_min
    return new_value

### 1. Digital payments penetration

In [15]:
indicators[0]

'Online-Service-Index (OSI)'

In [16]:
# load data
indicator = indicators[0]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Online-Service-Index (OSI)
e_government_index


In [17]:
df.head()

,Survey Year,Country Name,E-Government Rank,E-Government Index,E-Participation Index,Online Service Index,Human Capital Index,Telecommunication Infrastructure Index
0,2020,Iraq,143,0.4360,0.3095,0.3353,0.4358,0.5370
1,2020,Ireland,27,0.8433,0.8571,0.7706,0.9494,0.8100
2,2020,Israel,30,0.8361,0.7143,0.7471,0.8924,0.8689
3,2020,Italy,37,0.8231,0.8214,0.8294,0.8466,0.7932
4,2020,Jamaica,114,0.5392,0.3690,0.3882,0.7142,0.5151


In [18]:
# all data from 2020
df['Survey Year'].value_counts()

2020    193
Name: Survey Year, dtype: int64

In [19]:
# score looks like the one to use
df.describe()

,Survey Year,E-Government Rank,E-Government Index,E-Participation Index,Online Service Index,Human Capital Index,Telecommunication Infrastructure Index
count,193.0,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000
mean,2020.0,97.000000,0.598767,0.567723,0.561961,0.687992,0.546354
std,0.0,55.858452,0.214869,0.259592,0.249874,0.194440,0.259358
min,2020.0,1.000000,0.087500,0.000000,0.000000,0.000000,0.000000
25%,2020.0,49.000000,0.432000,0.357100,0.352900,0.559900,0.349600
50%,2020.0,97.000000,0.612900,0.571400,0.576500,0.739500,0.566900
75%,2020.0,145.000000,0.779800,0.797600,0.764700,0.841400,0.772300
max,2020.0,193.000000,0.975800,1.000000,1.000000,1.000000,1.000000


In [20]:
# df.Indicator.unique()

In [21]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Online Service Index'] 


min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))



In [22]:

# output scores to csv
df[['Country Name','Indicator','data_col','new_rank_score','higher_is_better']].to_csv('../indicator_scores/government_{}_scores.csv'.format(bf), index=False)

In [23]:
df[['Country Name','Indicator','data_col','new_rank_score','higher_is_better']].head(15)


,Country Name,Indicator,data_col,new_rank_score,higher_is_better
0,Iraq,Online-Service-Index (OSI),0.3353,2.6765,True
1,Ireland,Online-Service-Index (OSI),0.7706,4.8530,True
2,Israel,Online-Service-Index (OSI),0.7471,4.7355,True
3,Italy,Online-Service-Index (OSI),0.8294,5.1470,True
4,Jamaica,Online-Service-Index (OSI),0.3882,2.9410,True
5,Japan,Online-Service-Index (OSI),0.9059,5.5295,True
6,Jordan,Online-Service-Index (OSI),0.3588,2.7940,True
7,Kazakhstan,Online-Service-Index (OSI),0.9235,5.6175,True
8,Kenya,Online-Service-Index (OSI),0.6765,4.3825,True
9,Kiribati,Online-Service-Index (OSI),0.4941,3.4705,True


## 2. E-Participation index


In [24]:
indicator = indicators[1]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

E-Participation index
e_government_index


In [25]:
df.head()

,Survey Year,Country Name,E-Government Rank,E-Government Index,E-Participation Index,Online Service Index,Human Capital Index,Telecommunication Infrastructure Index
0,2020,Iraq,143,0.4360,0.3095,0.3353,0.4358,0.5370
1,2020,Ireland,27,0.8433,0.8571,0.7706,0.9494,0.8100
2,2020,Israel,30,0.8361,0.7143,0.7471,0.8924,0.8689
3,2020,Italy,37,0.8231,0.8214,0.8294,0.8466,0.7932
4,2020,Jamaica,114,0.5392,0.3690,0.3882,0.7142,0.5151


In [26]:
df['Survey Year'].unique()

array([2020], dtype=int64)

In [27]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['E-Participation Index'] 


min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))


# output scores to csv
df[['Country Name','Indicator','data_col','new_rank_score','higher_is_better']].to_csv('../indicator_scores/government_{}_scores.csv'.format(bf), index=False)

## 3. Use of public services online (% of services online, penetration, frequency of use)


In [28]:
indicator = indicators[2]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Use of public services online (% of services online, penetration, frequency of use)
digital_public_service_use


In [29]:
# drop first row
df = df.iloc[1: , :]

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 1 to 31
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Country             31 non-null     object
 1   Net Perception (%)  31 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 628.0+ bytes


In [31]:
df.head(15)

,Country,Net Perception (%)
1,UAE,61
2,Saudi Arabia,59
3,Singapore,54
4,China,53
5,New Zealand,52
6,Netherlands,51
7,Qatar,51
8,Canada,40
9,Denmark,48
10,India,45


In [32]:
# clean data to remove % and convert to float
df['Unnamed: 1'] = df['Unnamed: 1'].str.replace('%','')

df['Unnamed: 1'] = df['Unnamed: 1'].astype(float)

KeyError: 'Unnamed: 1'

In [ ]:
# create the standard columns
df['higher_is_better'] = True
df['Indicator'] =  indicator
df['Country Name'] = df['Net perception that needs are being met by digital services']
# df['2019 [YR2019]'] = df['2019 [YR2019]'].astype(float)
df['data_col'] = df['Unnamed: 1']

In [ ]:
# convert 1-100 %  into 1-6
min_rank = df['data_col'].min()
max_rank = df['data_col'].max()
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row, old_min=min_rank,old_max=max_rank))

df.sort_values(by='new_rank_score', ascending=False)

# prepare output
df = df[['Country Name', 'Indicator', 'data_col', 'new_rank_score','higher_is_better']]

# output scores
df.to_csv('../indicator_scores/government_{}_scores.csv'.format(bf), index=False)

In [ ]:
df.describe()

## 4. What is the % change of government digitizing public services?



In [ ]:
indicator = indicators[3]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

In [ ]:
df.head()

In [ ]:
df['GTMI > Correlation of indicators (without standardization)'].unique()

In [ ]:
df[['GovTech Index','data_country','data_year']].head()

In [ ]:
df.columns.tolist()

In [ ]:
df[['data_country', 'data_year']].info()

In [ ]:
# dcol = 'Percentage of total trade in services'
# indicol = 'Category Label'
# cname = 'Economy Label'

# # filter most recent year
# df = df[(df.Year==2019)&(df['Flow Label']=='Exports')]

# # create standard columns
# df['higher_is_better'] = True
# df['Indicator'] =  df[indicol]+' '+ df['Flow Label']
# df['Country Name'] = df[cname]
# # df['2019 [YR2019]'] = df['2019 [YR2019]'].astype(float)
# df['data_col'] = df[dcol]


# # convert 1-100 %  into 1-6
# df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row, old_min=0,old_max=100))

# df.sort_values(by='new_rank_score', ascending=False)

# # prepare output
# df = df[['Country Name', 'Indicator', 'data_col', 'new_rank_score','higher_is_better']]

# # output scores
# df.to_csv('../indicator_scores/{}_scores.csv'.format(bf+'_exports'), index=False)

## 5. Security incidents (# of relevant issues)



In [ ]:
indicator = indicators[4]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

In [ ]:
df['Country Characteristics'].unique()

In [ ]:
df.head()

In [ ]:
# create a rank from the number of attacks fields
df['data_rank'] = df['Number of Significant Cyberattacks (2006-2020)'].rank(method='max')

In [ ]:
df.data_rank.max()

In [ ]:
# create standard columns
df.rename(columns={'Country':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['data_rank'] 


min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 1-20 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

# need to invert since higher rank is not better
df['new_rank_score'] = (6-df['new_rank_score'])+1

# output scores to csv
df[['Country Name','Indicator','data_col','new_rank_score','higher_is_better']].to_csv('../indicator_scores/government_{}_scores.csv'.format(bf), index=False)

In [ ]:
df

## 6. ICT investment as a percentage of GDP


In [ ]:
indicator = indicators[5]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.SUBJECT.unique()

In [ ]:
df.TIME.max()

In [ ]:
df.Value.describe()

In [ ]:
dcol = 'Value'
indicol = indicator
cname = 'LOCATION'

# filter most recent year
df = df[(df.TIME==2010)]

# create standard columns
df['higher_is_better'] = True
df['Indicator'] =  indicator
df['Country Code'] = df[cname]
# # df['2019 [YR2019]'] = df['2019 [YR2019]'].astype(float)
df['data_col'] = df[dcol]

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 1-20 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

# df.sort_values(by='new_rank_score', ascending=False)

# # prepare output
df = df[['Country Code', 'Indicator', 'data_col', 'new_rank_score','higher_is_better']]

# # output scores
df.to_csv('../indicator_scores/government{}_scores.csv'.format(bf), index=False)

In [ ]:
df

## 7. R&D spending (% of GDP)




In [33]:
indicator = indicators[6]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

R&D spending (% of GDP)
RD_Percentage_GDP


In [35]:
df.head(100)

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2017,2018,2019,2020,Country Name,Country Code,Indicator Name,Indicator Code,data_country,data_year
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Aruba,ABW,Research and development expenditure (% of GDP),GB.XPD.RSDV.GD.ZS,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Africa Eastern and Southern,AFE,Research and development expenditure (% of GDP),GB.XPD.RSDV.GD.ZS,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Afghanistan,AFG,Research and development expenditure (% of GDP),GB.XPD.RSDV.GD.ZS,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Africa Western and Central,AFW,Research and development expenditure (% of GDP),GB.XPD.RSDV.GD.ZS,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Angola,AGO,Research and development expenditure (% of GDP),GB.XPD.RSDV.GD.ZS,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,High income,HIC,Research and development expenditure (% of GDP),GB.XPD.RSDV.GD.ZS,NaN,NaN
96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.79925,0.86096,NaN,NaN,"Hong Kong SAR, China",HKG,Research and development expenditure (% of GDP),GB.XPD.RSDV.GD.ZS,NaN,NaN
97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.04000,NaN,NaN,NaN,Honduras,HND,Research and development expenditure (% of GDP),GB.XPD.RSDV.GD.ZS,NaN,NaN
98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Heavily indebted poor countries (HIPC),HPC,Research and development expenditure (% of GDP),GB.XPD.RSDV.GD.ZS,NaN,NaN


In [ ]:
# min_rank = 1
# max_rank = df['Country Name'].nunique()

# # transform 1-24 rank into 1-6
# df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

# # need to invert score since higher rank is not better 
# # df['new_rank_score'] = (6-df['new_rank_score'])+1

In [ ]:
# # prepare output
# df.sort_values(by='new_rank_score', ascending=False)

# # prepare output
# df = df[['Country Name', 'Indicator', 'data_col', 'new_rank_score','higher_is_better']]

# # output scores
# df.to_csv('../indicator_scores/{}_scores.csv'.format(bf), index=False)

## 8. Evidence of digital strategies in/across Minstries



In [ ]:
indicator = indicators[7]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

In [ ]:
df.head()

In [ ]:
# # filter on relevant years
# df = df[(df['Time']==2017)&(df['Information and communication technologies']=='ICT-intensive')]

# # convert rank into 1-6 - in order to get most recent data 
# df['data_col'] = df['Value'] 

# # going to assume index is between 1-100 but not 100% sure
# min_rank = 0 #df['data_col'].min()
# max_rank = 100 #df['data_col'].max()

# # transform 1-147 rank into 1-6
# df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

# # need to invert score since higher rank is not better 
# # df['new_rank_score'] = (6-df['new_rank_score'])+1

# # df_rank[['Country ISO3', 'Country Name','Indicator','data_col','new_rank_score']].head()

# df.rename(columns={'Country':'Country Name'}, inplace=True)
# df['Indicator'] = indicator
# df['higher_is_better'] = True

# df[['Country Name','Indicator','data_col','new_rank_score','higher_is_better']].head(15)


# # output scores to csv
# df[['Country Name','Indicator','data_col','new_rank_score','higher_is_better']].to_csv('../indicator_scores/{}_scores.csv'.format(bf), index=False)

## 9. Evidence of focus on vulnerable groups


In [ ]:
indicator = indicators[8]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

In [ ]:
df.head()

## 10. % of digital skills certifications / training courses completed


In [ ]:
indicator = indicators[9]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

In [ ]:
df

In [ ]:
dcol = 'Global Rank'
indicol = indicator
cname = 'Country and Region'

# filter most recent year
# df = df[(df.TIME==2010)]

# create standard columns
df['higher_is_better'] = True
df['Indicator'] =  indicator
df['Country Name'] = df[cname]
# # df['2019 [YR2019]'] = df['2019 [YR2019]'].astype(float)
df['data_col'] = df[dcol]

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 1-20 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

# # need to invert score since higher rank is not better 
df['new_rank_score'] = (6-df['new_rank_score'])+1

df.sort_values(by='new_rank_score', ascending=False)

# # prepare output
df = df[['Country Name', 'Indicator', 'data_col', 'new_rank_score','higher_is_better']]

# # output scores
df.to_csv('../indicator_scores/government{}_scores.csv'.format(bf), index=False)

In [ ]:
df

## 11. % of government encrypted web traffic


In [ ]:
indicator = indicators[10]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
bnames

## 12. % of country encrypted web traffic

In [ ]:
indicator = indicators[11]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

### Score Aggregating

In [ ]:
import os


In [ ]:
# get list of files in scores folder
scores = os.listdir('../indicator_scores/')
scores = [s for s in scores if s.startswith('government')]

In [ ]:
scores

In [ ]:
# create a dataframe that concatenates all these file into one table
df = pd.concat([pd.read_csv('../indicator_scores/{}'.format(s)) for s in scores])    

In [ ]:
df

In [ ]:
# Data cleaning
df['new_rank_score'] = df['new_rank_score'].fillna(0)
df.sort_values(by=['Country Name'], ascending=True, inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df.info()

In [ ]:
df.head(15)

In [ ]:
df.describe()

In [ ]:
# checking country names
# sorted(df['Country Name'].unique().tolist())

In [ ]:
# remove trailing whitespaces from country name
df['Country Name'] = df['Country Name'].str.strip()


In [ ]:
df.head()

In [ ]:
# checking country names
# sorted(df['Country Name'].unique().tolist())

In [ ]:
# average indicator scores per country
agg_df = df.groupby(['Country Name']).agg({'new_rank_score':'mean','data_col':'count'})

In [ ]:
agg_df.columns = ['agg_score', 'count_source' ]

In [ ]:
max_number_sources = agg_df.describe()['count_source']['max']

In [ ]:
agg_df['agg_score_wt'] = agg_df['agg_score']*(agg_df['count_source']/max_number_sources)

In [ ]:
agg_df.sort_values(by='agg_score', ascending=False, inplace=True)

In [ ]:
agg_df.head(25)

In [ ]:
agg_df.to_csv('../pillar_scores/government_scores_v0.csv')